# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [3]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = pd.read_csv(file_path)
df = df.dropna(axis='columns', how='all')
df = df.dropna()
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]
df['int_rate'] = df['int_rate'].astype('str').str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100
x = {'Current': 'low_risk'}   
df = df.replace(x)
x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)
df.reset_index(inplace=True, drop=True)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.001719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.002000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.002000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.001640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.001474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [4]:
df_encoded = pd.get_dummies(df, columns=[ 
    'initial_list_status', 
    'home_ownership', 
    'verification_status', 
    'issue_d', 
    'pymnt_plan', 
    'next_pymnt_d', 
    'application_type', 
    'hardship_flag', 
    'debt_settlement_flag']) 

df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.001719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
1,25000.0,0.002000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,0,0,1,1,0,1,1,0,1,1
2,20000.0,0.002000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,0,0,1,1,0,1,1,0,1,1
3,10000.0,0.001640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,0,0,1,1,0,1,1,0,1,1
4,22000.0,0.001474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,0,0,1,1,0,1,1,0,1,1


## Split the Data into Training and Testing

In [5]:
# Create our features
X = df_encoded.drop(columns="loan_status", axis=1)

# Create our target
y = df["loan_status"]

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,pymnt_plan_n,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.001277,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.371696,0.451066,0.177238,1.0,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.000481,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.483261,0.497603,0.381873,0.0,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.000600,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.000881,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.001180,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,1.0,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.001557,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.003084,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [7]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

**To get similar results use `random_state=1`**

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_train.shape

(51612, 95)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [9]:
# Create the StandardScaler instance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset

X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [33]:
# Resample the training data with the BalancedRandomForestClassifier

from imblearn.ensemble import BalancedRandomForestClassifier

brfc_model = BalancedRandomForestClassifier(n_estimators =1000, random_state=1)
brfc = brfc_model.fit(X_train_scaled, y_train)

In [37]:
# Calculated the balanced accuracy score

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

predictions = brfc.predict(X_test_scaled)
acc_score = accuracy_score(y_test, predictions)
print(acc_score)

0.9212438244696309


In [38]:
# Display the confusion matrix

confusion_matrix(y_test, predictions)

array([[   61,    26],
       [ 1329, 15789]], dtype=int64)

In [39]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.70      0.92      0.08      0.80      0.63        87
   low_risk       1.00      0.92      0.70      0.96      0.80      0.66     17118

avg / total       0.99      0.92      0.70      0.95      0.80      0.66     17205



In [18]:
# List the features sorted in descending order by feature importance

importances = brfc.feature_importances_

sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

[(0.06752310729109846, 'total_rec_prncp'),
 (0.06305446866981336, 'total_rec_int'),
 (0.061343602918319036, 'total_pymnt'),
 (0.058459164568765536, 'last_pymnt_amnt'),
 (0.05776658086027447, 'total_pymnt_inv'),
 (0.026566695692226517, 'int_rate'),
 (0.02133660283619999, 'issue_d_Jan-2019'),
 (0.017059706437959683, 'dti'),
 (0.016793742136999454, 'installment'),
 (0.01678426020088416, 'annual_inc'),
 (0.01627818186538979, 'out_prncp'),
 (0.01605990077709895, 'mths_since_recent_inq'),
 (0.015542992712359599, 'out_prncp_inv'),
 (0.015533688827299906, 'il_util'),
 (0.015369069231905981, 'bc_util'),
 (0.01525111093002703, 'revol_bal'),
 (0.015046323008982813, 'max_bal_bc'),
 (0.014608836366712875, 'mo_sin_old_rev_tl_op'),
 (0.014228014188182798, 'avg_cur_bal'),
 (0.01416150634871667, 'tot_hi_cred_lim'),
 (0.013760746960257142, 'all_util'),
 (0.013642470670730659, 'mo_sin_old_il_acct'),
 (0.013563193472669093, 'bc_open_to_buy'),
 (0.013406794090884536, 'tot_cur_bal'),
 (0.013249470219988362,

### Easy Ensemble Classifier

In [40]:
# Train the Classifier

from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1, random_state=1, 
                                   replacement=False, sampling_strategy='auto', verbose=0, 
                                   warm_start=False)

model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

EasyEnsembleClassifier()

EasyEnsembleClassifier()

In [41]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test_scaled)
acc_score2 = accuracy_score(y_test, y_pred)
print(acc_score)

0.9212438244696309


In [42]:
# Display the confusion matrix

confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  989, 16129]], dtype=int64)

In [43]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.94      0.14      0.92      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.92      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.92      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Not sure what I did wrong, but I am getting the same number for the balanced accuracy score for both models.
2. Which model had the best recall score?

    Easy Ensemble Classifier

3. Which model had the best geometric mean score?

    Easy Enseble Classifier

4. What are the top three features?

    The top three features were: Total Received Principal (0.067), Total Received Interest (0.063), Last Payment Amount (0.061) 